In [97]:
import numpy as np
from motifcluster import clustering as mccl
from motifcluster import motifadjacency as mcmo
from motifcluster import utils as mcut

In [98]:
import torch
from torch.nn import Linear
import torch.nn.functional as F
import torch_geometric
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
# NormalizeFeatures()进行节点特征归一化，使特征总和为1
dataset = Planetoid(root='dataset',name='Cora',transform=NormalizeFeatures())

print(f'dataset:{dataset}')
print('====================')
print(f'Number of graphs:{len(dataset)}')
print(f'Number of features:{dataset.num_features}')
print(f'Number of classes:{dataset.num_classes}')

# 得到第一个graph对象
data = dataset[0]

print()
print(data)
print('=====================')
print(f'Number of nodes:{data.num_nodes}')
print(f'Number of edges:{data.num_edges}')
print(f'Average node degree:{data.num_edges/data.num_nodes:.2f}')
print(f'Number of training nodes:{data.train_mask.sum()}')
print(f'Training node label rate:{int(data.train_mask.sum())/data.num_nodes:.2f}')
print(f'Contains isolated nodes:{data.contains_isolated_nodes()}')
print(f'Contains self-loops:{data.contains_self_loops()}')
print(f'Is undirected:{data.is_undirected()}')

dataset:Cora()
Number of graphs:1
Number of features:1433
Number of classes:7

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Number of nodes:2708
Number of edges:10556
Average node degree:3.90
Number of training nodes:140
Training node label rate:0.05
Contains isolated nodes:False
Contains self-loops:False
Is undirected:True


D:\code\conda\anaconda\envs\pyg\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'contains_isolated_nodes' is deprecated, use 'has_isolated_nodes' instead
  warnings.warn(out)
D:\code\conda\anaconda\envs\pyg\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'contains_self_loops' is deprecated, use 'has_self_loops' instead
  warnings.warn(out)


In [99]:
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self,hidden_channels):
        super(GCN,self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_features,hidden_channels)
        self.conv2 = GCNConv(hidden_channels,dataset.num_classes)

    #     # 加入边权重的
    def forward(self,x,edge_index,edge_weight):
        x = self.conv1(x,edge_index,edge_weight)
        x = x.relu()
        x = F.dropout(x,p=0.5,training=self.training)
        x = self.conv2(x,edge_index,edge_weight)
        return x

# 未加入边权重的
#     def forward(self,x,edge_index):
#         x = self.conv1(x,edge_index)
#         x = x.relu()
#         x = F.dropout(x,p=0.5,training=self.training)
#         x = self.conv2(x,edge_index)
#         return x

model = GCN(hidden_channels=16)
print(model)

GCN(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 7)
)


In [100]:
# 使用MAM
adj = data.edge_index
print(data.edge_weight)
adj = torch_geometric.utils.to_scipy_sparse_matrix(data.edge_index)
mam = mcmo.build_motif_adjacency_matrix(adj,motif_name='M4',motif_type='func',mam_method='sparse',mam_weight_type='mean')
mam1 = mcmo.build_motif_adjacency_matrix(adj,motif_name='M13',motif_type='struc',mam_method='sparse',mam_weight_type='mean')
# mam_edge_index,mam_edge_weight = torch_geometric.utils.from_scipy_sparse_matrix(mam)
# print(mam_edge_weight)
# 加上原邻接矩阵与边权重
inte_adj = adj + mam + mam1
mam_edge_index,mam_edge_weight = torch_geometric.utils.from_scipy_sparse_matrix(inte_adj)
print(mam_edge_weight)

None
tensor([ 1.,  1.,  1.,  ..., 34.,  4.,  5.], dtype=torch.float64)


In [101]:
# GCN的训练
model = GCN(hidden_channels=16)
model = model.float()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01,weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# model, data = model.to(device), data.to(device)

def train():
    model.train()
    optimizer.zero_grad()
    # out = model(data.x,mam_edge_index)
    out = model(data.x.float(),mam_edge_index,mam_edge_weight.float())
    loss = criterion(out[data.train_mask],data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss

for epoch in range(1,201):
    loss = train()
    print(f'Epoch:{epoch:03d},Loss:{loss:.2f}')

Epoch:001,Loss:1.95
Epoch:002,Loss:1.94
Epoch:003,Loss:1.94
Epoch:004,Loss:1.93
Epoch:005,Loss:1.93
Epoch:006,Loss:1.92
Epoch:007,Loss:1.92
Epoch:008,Loss:1.91
Epoch:009,Loss:1.90
Epoch:010,Loss:1.90
Epoch:011,Loss:1.89
Epoch:012,Loss:1.88
Epoch:013,Loss:1.88
Epoch:014,Loss:1.87
Epoch:015,Loss:1.86
Epoch:016,Loss:1.86
Epoch:017,Loss:1.84
Epoch:018,Loss:1.85
Epoch:019,Loss:1.84
Epoch:020,Loss:1.83
Epoch:021,Loss:1.80
Epoch:022,Loss:1.81
Epoch:023,Loss:1.80
Epoch:024,Loss:1.79
Epoch:025,Loss:1.80
Epoch:026,Loss:1.78
Epoch:027,Loss:1.76
Epoch:028,Loss:1.74
Epoch:029,Loss:1.75
Epoch:030,Loss:1.74
Epoch:031,Loss:1.72
Epoch:032,Loss:1.70
Epoch:033,Loss:1.70
Epoch:034,Loss:1.70
Epoch:035,Loss:1.68
Epoch:036,Loss:1.66
Epoch:037,Loss:1.63
Epoch:038,Loss:1.63
Epoch:039,Loss:1.61
Epoch:040,Loss:1.63
Epoch:041,Loss:1.59
Epoch:042,Loss:1.60
Epoch:043,Loss:1.56
Epoch:044,Loss:1.56
Epoch:045,Loss:1.56
Epoch:046,Loss:1.51
Epoch:047,Loss:1.49
Epoch:048,Loss:1.48
Epoch:049,Loss:1.51
Epoch:050,Loss:1.47


In [102]:
# GCN的测试
def test():
    model.eval()
    # out = model(data.x,data.edge_index)
    out = model(data.x.float(),mam_edge_index,mam_edge_weight.float())
    pred = out.argmax(dim=1)
    test_correct = pred[data.test_mask] == data.y[data.test_mask]
    test_acc = int(test_correct.sum())/int(data.test_mask.sum())
    return test_acc

test_acc = test()
print(f'Test Accuracy:{test_acc:.6f}')

Test Accuracy:0.776000
